In [1]:
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [ ]:
original = cv.imread('./embriaoteste3.png')
cinza = cv.cvtColor(original, cv.COLOR_BGR2GRAY)
cinza_h = cinza.shape[0]
cinza_w = cinza.shape[1]

In [ ]:
def gaussModif(gama_L, gama_H, c, D_0, imagem):
   # Variáveis de interesse
     im_h = imagem.shape[0]
     im_w = imagem.shape[1]
   # Coordenadas do centro
     u_c = im_h/2
     v_c = im_w/2
     H = np.zeros(imagem.shape, dtype=’float’)
   # Matriz de coordenadas
     u, v = np.meshgrid(range(im_h), range(im_w), indexing=’ij’)
   # Etapas de cálculo do filtro gaussiano modificado
     D_uv_2 = (((u-u_c)**2 + (v-v_c)**2)**2).astype(‘float’)
     div_D0 = D_uv_2/(D_0**2)
     c_div_D0 = (-1)* c * div_D0
     exp_div_D0 = np.exp(c_div_D0)
     um_menos_exp = 1 — exp_div_D0
     mult_delta_gama = (gama_H — gama_L) * um_menos_exp
     H = gama_L + mult_delta_gama
     return H
# Função para adicionar pixels na imagem de forma a otimizar o cálculo da DFT/FFT
def zeroPadding(imagem):
     cinza_h2 = cv.getOptimalDFTSize(imagem.shape[0])
     cinza_w2 = cv.getOptimalDFTSize(imagem.shape[1])
     imagem_padded = np.zeros((cinza_h2, cinza_w2), dtype=’uint8')
     imagem_padded[0:cinza_h, 0:cinza_w] = imagem
     return imagem_padded
# Restaura a imagem ao tamanho original
def unpaddingImage(imagem_padded, imagem):
     temp = np.zeros(imagem.shape, dtype=’uint8')
     temp[0:imagem.shape[0], 0:imagem.shape[1]] =   imagem_padded[0:imagem.shape[0], 0:imagem.shape[1]]
     return temp
 
# Plota FFT da imagem
def plotaFFT(imagem):
     I_fft = np.fft.fft2(imagem)
     I_shift = np.fft.fftshift(I_fft)
     I_mag = np.abs(I_shift)
     I_log = np.log(1+ I_mag)
     I_int = np.uint8(I_log)
     I_norm = cv.normalize(I_int, None, 0, 255, norm_type=cv.NORM_MINMAX)
     cv.imshow(“TF”, I_norm)
# Aplica o filtro homomórfico
def aplicaFiltro(func_transf, imagem):
# Redistribui os pixels do filtro(descentralização) pois a função
# gaussModif() retorna os valores das baixas frequência no centro da
# imagem plotada
     h = np.fft.fftshift(func_transf)
# Imagem no domínio logarítmico: soma-se 1 à imagem para evitar a indefinição do logaritmo
     I_log = np.log(1 + imagem)
# FFT
     I_fft = np.fft.fft2(I_log)
# Filtragem
     I_filt = h * I_fft
# IFFT
     I_inv_TF = np.fft.ifft2(I_filt)
# Exponencial: subtração do pixel adicionado na etapa do logaritmo
     I_exp = np.exp(np.real(I_inv_TF)) — 1
# Converte para uint8
     I_int = np.uint8(I_exp)
     return I_int